In [1]:
!pip3 install google-api-python-client

In [2]:
from apiclient.discovery import build
# from apiclient.errors import HttpError
# from oauth2client.tools import argparser

In [3]:
import json
with open('secret.json') as f:
    secret = json.load(f)

In [13]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# def youtube_search(options):
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

q = 'Python 自動化'
max_results = 30

response = youtube.search().list(
    q=q,
    part='id,snippet',
    order='viewCount',
    type='video',
    maxResults=max_results
).execute()

In [15]:
items = response['items']
item = items[0]
item

{'kind': 'youtube#searchResult',
 'etag': 'yPLe0hCKJFyP4cO3flm11RXqm-w',
 'id': {'kind': 'youtube#video', 'videoId': '-5v7CuXm7Ns'},
 'snippet': {'publishedAt': '2020-05-20T11:41:43Z',
  'channelId': 'UCGlgXjYVoHLD86TQQ799WIw',
  'title': 'Pythonで面倒なExcelの仕事を自動化しよう( 第一弾 )｜一瞬で仕事がおわるプログラミング活用術',
  'description': 'この動画では、PythonによるExcel作業の自動化ついては説明をしました。細かいコードの説明はPandas超入門コースでする予定です。この動画では、どんなことができる ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/-5v7CuXm7Ns/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/-5v7CuXm7Ns/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/-5v7CuXm7Ns/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'キノコード / プログラミング学習チャンネル',
  'liveBroadcastContent': 'none',
  'publishTime': '2020-05-20T11:41:43Z'}}

In [17]:
video_id = item['id']['videoId']

In [19]:
channel_id = item['snippet']['channelId']

In [23]:
import pandas as pd

In [24]:
items_id = []
items = response['items']
for item in items:
    item_id = {}
    item_id['video_id'] = item['id']['videoId']
    item_id['channel_id'] = item['snippet']['channelId']
    items_id.append(item_id)

df_video = pd.DataFrame(items_id)

In [26]:
df_video[:3]

,video_id,channel_id
0,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw
1,LgZ8Li97yoM,UCGlgXjYVoHLD86TQQ799WIw
2,JNKZYhyr9Yw,UC6M7Qk2It63qD0jhiKgLWZA


In [30]:
def video_search(youtube, q='自動化', max_results=50):

    response = youtube.search().list(
        q=q,
        part='id,snippet',
        order='viewCount',
        type='video',
        maxResults=max_results
    ).execute()

    items_id = []
    items = response['items']
    for item in items:
        item_id = {}
        item_id['video_id'] = item['id']['videoId']
        item_id['channel_id'] = item['snippet']['channelId']
        items_id.append(item_id)

    df_video = pd.DataFrame(items_id)
    
    return df_video

In [31]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

df_video = video_search(youtube, q='Python 自動化', max_results=50)

In [32]:
df_video[:3]

,video_id,channel_id
0,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw
1,mIYJrEScAGI,UCMMjv61LfBy5J3AT8Ua0NGQ
2,LgZ8Li97yoM,UCGlgXjYVoHLD86TQQ799WIw


In [36]:
channel_ids = df_video['channel_id'].unique().tolist()
channel_ids[:3]

['UCGlgXjYVoHLD86TQQ799WIw',
 'UCMMjv61LfBy5J3AT8Ua0NGQ',
 'UC6M7Qk2It63qD0jhiKgLWZA']

In [41]:
subscriber_list = youtube.channels().list(
    id=','.join(channel_ids),
    part='statistics',
    fields='items(id,statistics(subscriberCount))'
).execute()

In [45]:
subscriber_list['items'][:5]

[{'id': 'UCGFxqqRaM97VjM5ToRfkoFg', 'statistics': {'subscriberCount': '1600'}},
 {'id': 'UC1uj19dZQXa17-SqNGpVNLw',
  'statistics': {'subscriberCount': '14900'}},
 {'id': 'UCYPkI8lgfiNXmGxCglZhOwA', 'statistics': {'subscriberCount': '500'}},
 {'id': 'UCEj6hquMBUiQGunwIO1zVZA',
  'statistics': {'subscriberCount': '21800'}},
 {'id': 'UCaeuEZcPrQMdMvI6VMFHc5Q', 'statistics': {'subscriberCount': '4560'}}]

In [91]:
subscribers = []
for item in subscriber_list['items']:
    subscriber = {}
    subscriber['channel_id'] = item['id']
    subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
    subscribers.append(subscriber)

df_subscribers = pd.DataFrame(subscribers)

In [92]:
df_subscribers[:3]

,channel_id,subscriber_count
0,UCGFxqqRaM97VjM5ToRfkoFg,1600
1,UC1uj19dZQXa17-SqNGpVNLw,14900
2,UCYPkI8lgfiNXmGxCglZhOwA,500


In [58]:
df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
df.head(3)

,video_id,channel_id,subscriber_count
0,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw,113000
1,LgZ8Li97yoM,UCGlgXjYVoHLD86TQQ799WIw,113000
2,HuJV9Vq6snQ,UCGlgXjYVoHLD86TQQ799WIw,113000


In [67]:
df_extracted = df[df['subscriber_count'] < 10000]
df_extracted[:3]

,video_id,channel_id,subscriber_count
18,2MQ3l5rQNKk,UCaeuEZcPrQMdMvI6VMFHc5Q,4560
19,0zelr_2KkrU,UCaeuEZcPrQMdMvI6VMFHc5Q,4560
32,nCfAtMooIQQ,UCNste53_VRtP6MGUfitOYQA,4210


In [66]:
video_ids = df_extracted['video_id'].tolist()

In [69]:
videos_list = youtube.videos().list(
    id=','.join(video_ids),
    part='snippet,statistics',
    fields='items(id,snippet(title),statistics(viewCount))'
).execute()

In [72]:
videos_list['items'][:3]

[{'id': '2MQ3l5rQNKk',
  'snippet': {'title': '【Python入門】Pythonでできること、学習方法についてまとめてみた'},
  'statistics': {'viewCount': '90130'}},
 {'id': '0zelr_2KkrU',
  'snippet': {'title': '【Pythonで自動化？】Seleniumでブラウザを自動で動かしてみよう！'},
  'statistics': {'viewCount': '9085'}},
 {'id': 'nCfAtMooIQQ',
  'snippet': {'title': 'その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】'},
  'statistics': {'viewCount': '34586'}}]

In [77]:
videos_info = []
items = videos_list['items']
for item in items:
    video_info = {}
    video_info['video_id'] = item['id']
    video_info['title'] = item['snippet']['title']
    video_info['view_count'] = item['statistics']['viewCount']
    videos_info.append(video_info)

df_videos_info = pd.DataFrame(videos_info)

In [79]:
df_videos_info[:3]

,video_id,title,view_count
0,2MQ3l5rQNKk,【Python入門】Pythonでできること、学習方法についてまとめてみた,90130
1,0zelr_2KkrU,【Pythonで自動化？】Seleniumでブラウザを自動で動かしてみよう！,9085
2,nCfAtMooIQQ,その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】,34586


In [81]:
results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
results[:3]

,video_id,channel_id,subscriber_count,title,view_count
0,2MQ3l5rQNKk,UCaeuEZcPrQMdMvI6VMFHc5Q,4560,【Python入門】Pythonでできること、学習方法についてまとめてみた,90130
1,0zelr_2KkrU,UCaeuEZcPrQMdMvI6VMFHc5Q,4560,【Pythonで自動化？】Seleniumでブラウザを自動で動かしてみよう！,9085
2,nCfAtMooIQQ,UCNste53_VRtP6MGUfitOYQA,4210,その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】,34586


In [83]:
results = results.loc[:,['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]

In [84]:
results[:3]

,video_id,title,view_count,subscriber_count,channel_id
0,2MQ3l5rQNKk,【Python入門】Pythonでできること、学習方法についてまとめてみた,90130,4560,UCaeuEZcPrQMdMvI6VMFHc5Q
1,0zelr_2KkrU,【Pythonで自動化？】Seleniumでブラウザを自動で動かしてみよう！,9085,4560,UCaeuEZcPrQMdMvI6VMFHc5Q
2,nCfAtMooIQQ,その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】,34586,4210,UCNste53_VRtP6MGUfitOYQA


In [113]:
def get_results(df_video, threshold=5000):
    channel_ids = df_video['channel_id'].unique().tolist()

    subscriber_list = youtube.channels().list(
        id=','.join(channel_ids),
        part='statistics',
        fields='items(id,statistics(subscriberCount))'
    ).execute()

    subscribers = []
    for item in subscriber_list['items']:
        subscriber = {}
        if len(item['statistics']) > 0:
            subscriber['channel_id'] = item['id']
            subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
        else:
            subscriber['channel_id'] = item['id']            
        subscribers.append(subscriber)

    df_subscribers = pd.DataFrame(subscribers)

    df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
    df_extracted = df[df['subscriber_count'] < threshold]

    video_ids = df_extracted['video_id'].tolist()
    videos_list = youtube.videos().list(
        id=','.join(video_ids),
        part='snippet,statistics',
        fields='items(id,snippet(title),statistics(viewCount))'
    ).execute()

    videos_info = []
    items = videos_list['items']
    for item in items:
        video_info = {}
        video_info['video_id'] = item['id']
        video_info['title'] = item['snippet']['title']
        video_info['view_count'] = item['statistics']['viewCount']
        videos_info.append(video_info)

    df_videos_info = pd.DataFrame(videos_info)

    results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
    results = results.loc[:,['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]
    return results

In [116]:
df_video = video_search(youtube, q='Python Excel', max_results=50)
results = get_results(df_video, threshold=1000)

In [117]:
results[:3]

,video_id,title,view_count,subscriber_count,channel_id
0,CzuwQwI5zk8,Data Analysis with Python for Excel User Part ...,100523,790,UC-gGhiNMqUTsrE7N9vDrR1Q
1,C2O3O_GydV4,Use Python and Pandas to Work With Excel,49500,426,UCAOvckbaZM8MEh9R83KH9HA
2,WlGq6R4U2gE,How to read Excel files with Python: A beginne...,44115,148,UC8AHh9YDeRX0WkfkHTVJPVA


In [120]:
video_id = ''

In [121]:
len(video_id)

0